In [62]:
import html
import re
import numpy as np

import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

from typing import Any
from nltk.tokenize import sent_tokenize
from nltk.cluster.util import cosine_distance

In [70]:
def preprocess_text(text: str) -> str:
    text = html.unescape(text)
    text = text.encode('ascii', 'ignore').decode('ascii')
    text = re.sub(' +', ' ', text.strip())
    return text

def extract_sentences(text: str) -> list:
    sents = sent_tokenize(text)
    return sents

def extract_embedding(sents: list, sess: Any) -> list:
    vectors = session.run(embeddings, feed_dict={text_ph: sents})
    return vectors

def get_cosine_similarity(vector1: list, vector2: list) -> float:
    return 1 - cosine_distance(vector1, vector2)
    
def get_similarity_matrix(sents: list, vectors: list) -> np.ndarray:        
    sm = np.zeros([len(sents), len(sents)])
    for idx1 in range(len(sents)):
        for idx2 in range(len(sents)):
            if idx1 == idx2:
                continue
            sm[idx1][idx2] = get_cosine_similarity(vectors[idx1], vectors[idx2])
    return sm

def run_text_rank_algo(d: float, epoch: int, N: int, M: np.ndarray, convergence_threshold: float=1e-5) -> np.ndarray:
    rank = np.ones(N)/N
    I = np.ones(N)
    
    prev_rank = rank
    for i in range(epoch):
        if i==0:
            continue
        else:
            rank = d*(M@rank) + ((1-d)/N)*I  
            
        if abs(sum(prev_rank) - sum(rank)) <  convergence_threshold:
            break
        else:
            prev_rank = rank
            
    return rank

def get_top_sentences(number: int, sents: list, rank: np.ndarray) -> list:
    top_sentences = []

    sorted_rank = np.argsort(rank)
    sorted_rank = list(sorted_rank)
    sorted_rank.reverse()
        
    for i in range(number):
        sent = sents[sorted_rank[i]]
        top_sentences.append(sent)

    return top_sentences
    

In [64]:
print("Load pre-trained universal sentence encoder model....")
# embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
embed = hub.Module("/home/rahul/universal-sentence-encoder/2")
text_ph = tf.placeholder(tf.string)
embeddings = embed(text_ph)
print("Done!")

print("Creating tensorflow session...")
session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
print("Done!")








Load pre-trained universal sentence encoder model....
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Done!
Creating tensorflow session...
Done!


In [75]:
text_str = '''
It’s not that CO2 isn’t a problem — it’s the main problem. But on a molecule-for-molecule basis, methane traps more heat, so converting it into something less potent would reduce its climate impact.
In fact, by restoring the concentration of methane in the atmosphere to preindustrial levels, this counterintuitive strategy could eliminate about a sixth of human-caused warming, according to a paper published Monday in Nature Sustainability. And it would add only a few months’ worth of CO2 emissions to the atmosphere.
“In the grand scheme of carbon dioxide emissions, this would not be a deal-breaker,” said lead author Rob Jackson, an earth scientist at Stanford University.
Jackson, like most scientists, says the best strategy to combat climate change is to stop emitting greenhouse gases.
“Having said that, we’re not getting the job done on reducing emissions, so I think we need to look at some of these other approaches,” said Jackson, who chairs the Global Carbon Project, which tracks greenhouse gases.
Already, it’s clear that people will have to pull huge amounts of carbon dioxide out of the atmosphere to meet the goals of the Paris climate accord, which would limit global warming to less than 2 degrees Celsius above preindustrial levels.
Some scenarios call for removing up to 10 billion metric tons of the gas per year — a quarter of humanity’s annual emissions — by storing it in biomass or soil, or building facilities that directly capture the gas from the air.
“But no one’s talking about this for methane,” Jackson said. “That’s what we want to accomplish with this paper.”
Methane in the atmosphere is surging, and that’s got scientists worried.
Methane hasn’t caused as much warming as CO2, but humans have had a much bigger impact on the methane cycle, he said.
Over the last 200 years, we have more than doubled its concentration in the atmosphere by extracting fossil fuels, raising livestock, and allowing the gas to escape from landfills and wastewater treatment plants, among other activities. Today, methane levels are 1,860 parts per billion — and rising — compared with 750 ppb before 1800.
Methane also has a more acute warming effect. It traps about 84 times as much heat as CO2 over a 20-year period, and 28 times as much over the course of a century. (Its potency drops because it has a shorter lifetime in the atmosphere.) So turning a molecule of methane into a molecule of CO2 would slash its climate-altering capacity, Jackson and his coauthors say.
And the trade-off would be worth it, they argue. Restoring methane to its preindustrial concentration would involve a one-time conversion of 3.2 billion tons of the gas into CO2. That would increase CO2 levels — which were 280 ppm in preindustrial times and are currently 415 ppm — by less than 1 part per million.
“That’s one of the real selling points in my mind,” Jackson said.
So can it be done?
At this stage, the idea is mostly theoretical, but the authors are cautiously optimistic.
Researchers have started developing ways to oxidize methane into methanol, a valuable compound used for 
fuel and chemical manufacturing. The same reactions, if allowed to proceed further, could also be used 
to convert methane into carbon dioxide.
Edward Solomon, a Stanford chemist who worked on the new paper, studies one promising method of processing methane using minerals called zeolites. They catch the gas, and help with oxidation.
Researchers propose removing methane from the atmosphere by pulling air through large fans and using materials like zeolites to catalyze its conversion into carbon dioxide. Thousands of these arrays would be needed to restore preindustrial methane levels.
The researchers envision using these kinds of materials in facilities like those being developed to remove CO2, which use fans to draw air into chambers where the gases are captured through chemical reactions.
The low concentration of methane means you’d have to process a lot of air to reduce atmospheric levels, Jackson said. “It would take many thousands of these arrays to make a dent,” he said, although it would be a much smaller effort than what’s been proposed for dealing with CO2.
And the economics could prove attractive — if countries eventually settle on a price for carbon emissions, either through a tax or a cap-and-trade system like the one in California.
Indeed, removing methane could be many times more lucrative than removing CO2, because the value of keeping a greenhouse gas out of the atmosphere typically depends on its heat-trapping ability.
Imagine a system in which the market will pay $50 for every ton of CO2 emissions that can be avoided. In the 100-year scenario in which methane is considered to be 28 times more powerful, the value of eliminating one ton of it would be $1,400. There’d be a small cost for emitting a little bit of CO2 in the conversion process, but even so, the take home would be about $1,250. (A bipartisan bill introduced in the House this year would impose a fee on carbon starting at $15 per ton of CO2 and increasing to more than $100 by 2030; models predict prices could climb as high as $500 later in the century.)
At the upper end of that range, researchers estimate that a methane removal facility the size of a football field could generate millions of dollars of income.
But that could be dangerous, warned Myles Allen, a climate scientist at Oxford University in the U.K. who was not involved in the paper.
Allen fears that, if there’s money to be made — or saved — removing methane from the atmosphere, some emitters may favor doing that instead dealing with CO2.
“That might make good business sense, but it would be terrible for the climate,” he said.
That’s because methane and carbon dioxide affect the climate in different ways.
Methane only stays in the atmosphere for about a decade, so its climate effects are short-lived. CO2, on the other hand, lingers for centuries. Thus, removing some methane might yield a small benefit right now, but won’t help solve the climate problem in the long run, Allen said.
Yet if carbon pricing schemes treat the gases as if they are interchangeable, then companies could offset their CO2 emissions by reducing methane emissions.
“You can imagine the world’s airlines getting on to this and getting all excited because suddenly it looks like they’ve got a very cheap pass and they can just carry on emitting,” he said. “And of course, they’ll just carry on causing global warming.”
Still, Allen agreed that we have to reduce the concentration of methane in the atmosphere. And he endorses the idea of developing technologies to offset the methane emissions that are difficult to eliminate, as the researchers also suggest in the paper.
Rice grows in a watery field near the city of Williams in the Sacramento Valley. Flooded soils produce methane, and rice cultivation represents about 10% of human-caused emissions.
Rice grows in a watery field near the city of Williams in the Sacramento Valley. Flooded soils produce methane, and rice cultivation represents about 10% of human-caused emissions. (Brian van der Brug / Los Angeles Times)
Raising cattle, growing rice and other seemingly essential activities produce the gas and they are unlikely to go away. “It’s hard for me to see any time in the near future when methane emissions will be zero,” Jackson said.
He and his colleagues will keep working to make methane conversion technology a reality, and they hope their paper will encourage others to try as well.
Jackson said part of his motivation is symbolic. Resetting methane concentrations to their preindustrial levels offers a way to “repair the atmosphere,” he said. And that might be inspiring.
“The notion of restoring the atmosphere sends a message of hope to people,” he said. “I would love to see something like this happen in my lifetime.”

'''

print(text_str)


It’s not that CO2 isn’t a problem — it’s the main problem. But on a molecule-for-molecule basis, methane traps more heat, so converting it into something less potent would reduce its climate impact.
In fact, by restoring the concentration of methane in the atmosphere to preindustrial levels, this counterintuitive strategy could eliminate about a sixth of human-caused warming, according to a paper published Monday in Nature Sustainability. And it would add only a few months’ worth of CO2 emissions to the atmosphere.
“In the grand scheme of carbon dioxide emissions, this would not be a deal-breaker,” said lead author Rob Jackson, an earth scientist at Stanford University.
Jackson, like most scientists, says the best strategy to combat climate change is to stop emitting greenhouse gases.
“Having said that, we’re not getting the job done on reducing emissions, so I think we need to look at some of these other approaches,” said Jackson, who chairs the Global Carbon Project, which tracks gr

In [74]:
text_str = preprocess_text(text_str)
sents = extract_sentences(text_str)
vectors = extract_embedding(sents, session)

sm = get_similarity_matrix(sents, vectors)
rank = run_text_rank_algo(0.85, 10, len(sents), sm, 1e-5)
    
ranked_sents = get_top_sentences(5, sents, rank)

print("\nSummary:\n")
for i, val in enumerate(ranked_sents):
    print(val)


Summary:

Indeed, removing methane could be many times more lucrative than removing CO2, because the value of keeping a greenhouse gas out of the atmosphere typically depends on its heat-trapping ability.
Allen fears that, if theres money to be made or saved removing methane from the atmosphere, some emitters may favor doing that instead dealing with CO2.
Still, Allen agreed that we have to reduce the concentration of methane in the atmosphere.
Restoring methane to its preindustrial concentration would involve a one-time conversion of 3.2 billion tons of the gas into CO2.
And it would add only a few months worth of CO2 emissions to the atmosphere.


In [58]:
session.close()